In [238]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [239]:
monster_url = 'https://www.monster.fi/en/jobs?search=Data%20analyst&job_geo_location=Helsinki%2C%20Suomi&radius=50&Find_Jobs=Find%20Jobs&lat=60.16985569999999&lon=24.938379&country=Suomi&administrative_area_level_1=undefined'

In [240]:
html = requests.get(monster_url)

In [241]:
#response of the html request: 200 - successful, 404- page not found
print(html)

<Response [200]>


In [242]:
jobs = BeautifulSoup(html.content,'html.parser')

In [243]:
#print(jobs.prettify())

In [244]:
#using selector to select the page content
listings = jobs.select('div.view-content')[0]

In [245]:
#extracting the job name:
job_listings = listings.find_all('h2',class_='node__title')
name = [j.get_text().replace('\n','') for j in job_listings]

In [246]:
#extracting the application deadline:
job_date = listings.find_all('span',class_='date')
deadline = [d.get_text().replace('\n','').split(',')[0] for d in job_date]

In [247]:
#extracting the company name
job_place = listings.find_all('span',class_='recruiter-company-profile-job-organization')
organization = [p.get_text() for p in job_place]

In [248]:
#extracting the location of the job:
job_area = listings.find_all('div',class_='location')
place = [a.get_text().replace('\n','') for a in job_area]

In [249]:
#extracting the skills/domain needed for the job:
job_skills = listings.find_all('div', class_='terms')
expertise = [e.get_text().replace('\n      ','').split('|') for e in job_skills]

In [250]:
#extracting the job url to apply for the job:
links = listings.find_all('a',class_='recruiter-job-link')
job_links = [l.get('href') for l in links]
#getting the unique job urls:
job_links = pd.Series(job_links)
job_links = job_links.unique()

In [251]:
#creating a pandas dataframe with the attributes extracted from the page:
data_jobs = pd.DataFrame({'job_Name': name, 'application_date' : deadline, 'company': organization, 'location': place, 'domain': expertise , 'url': job_links})

In [252]:
#printing the head
data_jobs.head()

,job_Name,application_date,company,location,domain,url
0,Data Analyst,29.04.2022,Posti Oy,Helsinki,"[Manufacturing , Industrial Work , Software ...",https://www.monster.fi/en/job/data-analyst-108...
1,"Data Analyst, Konecranes, Espoo + remote",29.04.2022,Academic Work,Espoo - Distance: 16km,"[Software Development , Information Technolog...",https://www.monster.fi/en/job/data-analyst-kon...
2,Data Engineer,03.05.2022,Columbia Road,Helsinki,"[Engineering , Industrial Work , Software De...",https://www.monster.fi/en/job/data-engineer-11...
3,"BI Analyst, Group Financial Planning & Analysis",02.05.2022,Normet,Espoo - Distance: 16km,[Finance ],https://www.monster.fi/en/job/bi-analyst-group...
4,Data Scientist for Vattenfall,29.04.2022,Vattenfall Oy,Helsinki,"[Software Development , Information Technolog...",https://www.monster.fi/en/job/data-scientist-v...


In [253]:
#inspecting the size of our web scrape
data_jobs.shape

(20, 6)

In [255]:
#saving it to a csv file locally
data_jobs.to_csv('data_jobs.csv')